In [ ]:
import torch 
from torch import nn
from torch.nn import functional as F
import numpy as np
import pandas as pd
from vocab import Vocab
from dataprocessor import DataProcessor
from conv_model import ConvProtein
from utils import *
from tqdm import trange

In [ ]:
def get_data(filename):
    data = pd.read_csv(filename)
    return data['protein_sequence'].values , data['tm'].values 

trainx,trainy = get_data('./data/train_fixed.csv')
vocab = Vocab()
vocab.from_file()

vectors = load_ptvectors('./vectors.npy')
datafeeder = DataProcessor(vocab , trainx,trainy)


In [ ]:
model = ConvProtein(vectors,lr=1e-4)


for i in trange(1000):
    x,y = datafeeder.sample(100)
    model.train(x,y)